In [2]:
from fastai.vision.all import *
import cv2
import skimage as sm
import skimage.io
import tifffile
path = Path('/notebooks/EcadWatershedTrain')
# Training

valid_fnames = (path/'valid.txt').read_text().split('\n')
path_im = path/'dat/ecadBlur3_101/train_images'
path_lbl = path/'dat/ecadBlur3_101/train_masks'
codes = np.loadtxt(path/'codes.txt', dtype=str)
# codes = np.loadtxt(path/'codes.txt', dtype=str)
fnames = get_image_files(path_im)
lbl_names = get_image_files(path_lbl)
get_msk = lambda o: path/f'dat/ecadBlur3_101/train_masks'/f'mask{o.stem.split("g")[1]}{o.suffix}'

camvid = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                   get_items=get_image_files,
                   splitter=FileSplitter(path/'valid.txt'),
                   get_y=get_msk,
                   batch_tfms=[*aug_transforms(pad_mode='zeros'), Normalize.from_stats(*imagenet_stats)])

dls = camvid.dataloaders(path/f'dat/ecadBlur3_101/train_images', bs=4)
dls.vocab = codes
name2id = {v:k for k,v in enumerate(codes)}
void_code = name2id['Void']
def acc_camvid(inp, targ):
  targ = targ.squeeze(1)
  mask = targ != void_code
  return (inp.argmax(dim=1)[mask]==targ[mask]).float().mean() 
opt = ranger
learn = unet_learner(dls, resnet101, metrics=acc_camvid, self_attention=True, act_cls=Mish, opt_func=opt).to_fp16()

device="cuda"
learn.load(f'ecadBlur3Watershed_101')
model = learn.model
model = model.to(device)

RuntimeError: Error(s) in loading state_dict for DynamicUnet:
	size mismatch for layers.12.0.weight: copying a param with shape torch.Size([5, 291, 1, 1]) from checkpoint, the shape in current model is torch.Size([3, 291, 1, 1]).
	size mismatch for layers.12.0.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([3]).

In [ ]:
print(0)

In [ ]:
from cgitb import reset
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
import skimage as sm
import skimage.io
from matplotlib import pyplot as plt
import tifffile
from fastai.vision.all import *
from skimage.feature import blob, blob_dog, blob_log, blob_doh
from os.path import exists

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
NUM_WORKERS = 2
IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512
PIN_MEMORY = True

# util
    
def main():
    
    cwd = os.getcwd()
    filenames = os.listdir(cwd + "/dat/ecadBlur3/test_images")
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
    if ".ipynb_checkpoints" in filenames:
        filenames.remove(".ipynb_checkpoints")
    filenames.sort()

    label=0
    for filename in filenames:
        print(filename)
        out = learn.predict(f'dat/ecadBlur3/test_images/{filename}')[2][1]
#         print(out.shape)
        out = np.asarray(out*255, "uint8")
        tifffile.imwrite(f"dat_output/ecadBlur3_101/pred_{filename}", out, 
            imagej=True,)

In [ ]:
main()